# Charts of daily / weekly percentage changes and daily / weekly ranges (between min max)

In [1]:
import numpy as np
import glob
import pandas as pd
from bokeh.plotting import figure, show, gridplot
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from math import pi
output_notebook()

Loading BokehJS ...

In [2]:
def plot_change_range(in_df: pd.DataFrame, date: str) -> pd.DataFrame:
    df = in_df.copy()
    w = 12*60*60*1000 # half day in ms
    df = df.loc[df.index >= date]
    df["Change"] = (df["<CLOSE>"] / df["<CLOSE>"].shift(1)) * 100 - 100
    df["Range"] = (df["<HIGH>"] - df["<LOW>"]) / df["<CLOSE>"] * 100 
    inc = df["Change"] >= 0
    dec = df["Change"] < 0
    TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,save"

    pp = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, plot_height=400, title = df["<TICKER>"].values[0] + " Close change vs. t-1 [%]")
    ppv = figure(x_axis_type="datetime", x_range=pp.x_range, tools=TOOLS, plot_width=1000, plot_height=200, title="VOLUME")
    ppx = figure(x_axis_type="datetime", tools=TOOLS, plot_width=1000, plot_height=300, title = df["<TICKER>"].values[0] + " (Low-High range) / Close [%]")
    pp.xaxis.major_label_orientation = pi/4
    pp.xaxis.ticker.desired_num_ticks = 60
    pp.yaxis.ticker.desired_num_ticks = 5
    pp.grid.grid_line_alpha=0.3
    
    ppv.xaxis.major_label_orientation = pi/4
    ppv.xaxis.ticker.desired_num_ticks = 60
    ppv.yaxis.ticker.desired_num_ticks = 5
    ppv.grid.grid_line_alpha=0.3
    
    ppx.xaxis.major_label_orientation = pi/4
    ppx.xaxis.ticker.desired_num_ticks = 60
    ppx.yaxis.ticker.desired_num_ticks = 5
    ppx.grid.grid_line_alpha=0.3

    pp.vbar(df.index[inc], w, 0, df["Change"][inc], fill_color="#D5E1DD", line_color="black")
    pp.vbar(df.index[dec], w, 0, df["Change"][dec], fill_color="#F2583E", line_color="black")
    pp.add_tools(HoverTool(tooltips=[( 'Change [%]',  '$y')]))
    
    ppx.vbar(df.index, w, 0, df["Range"], fill_color="#D5E1DD", line_color="black")
    ppx.add_tools(HoverTool(tooltips=[( 'Range [%]',  '$y')]))
  
    ppv.vbar(df.index, w, 0, df["<VOL>"], fill_color="#F2583E", line_color="black")
    
    gg = gridplot([[pp], [ppv], [ppx]])
    show(gg)  # open a browser
    return df

In [3]:
path_basked = ['..\\data\\CDPROJEKT.mst',
               '..\\data\\CCC.mst',
               '..\\data\\11BIT.mst',
               '..\\data\\XTB.mst',
               '..\\data\\ALIOR.mst',
               '..\\data\\MBANK.mst',
               '..\\data\\MERCATOR.mst',
               '..\\data\\VRG.mst',
               '..\\data\\RAFAKO.mst']
date_start = "20191201"

for path in path_basked:
    data = pd.read_csv(path, index_col=1)
    data.index = pd.to_datetime(data.index, format="%Y%m%d")
    my_df = plot_change_range(data, date_start)

In [24]:
my_df[["Change", "<CLOSE>"]]

,Change,<CLOSE>
<DTYYYYMMDD>,,
2019-12-02,NaN,257.5
2019-12-03,-0.194175,257.0
2019-12-04,1.284047,260.3
2019-12-05,0.653093,262.0
2019-12-06,-0.267176,261.3
...,...,...
2020-05-18,3.799515,385.2
2020-05-19,-1.427830,379.7
2020-05-20,4.029497,395.0
